In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib3
from swiftshadow.classes import Proxy

In [9]:
# Importing dataframe
df = pd.read_excel('data/fonds.xlsx', )

In [10]:
df

,Fonds,Site internet
0,Andera Partners (Mid-cap),https://www.anderapartners.com/fr/
1,Ardian (Expansion),https://www.ardian.com/fr/expansion
2,Astorg (Mid-cap),https://www.astorg.com/
3,Bridgepoint (Mid-Cap),https://www.bridgepoint.eu/
4,Capza (Mid-Cap),https://capza.co/fr/
5,Carlyle (Tech),https://www.carlyle.com/
6,Five Arrows (Principal Investments),https://www.rothschildandco.com/en/five-arrows/
7,Gimv,https://www.gimv.com/fr
8,IK Partners (Small-Cap),https://ikpartners.com/
9,Keensight (Flagship),https://keensight.com/


In [11]:
# Disabling warnings about ssl certificated because we will be making requests without ssl (cause some website have expired ssl certificates we need to ignore)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [12]:
def scrape_company_text(url):
    try:
        # Fetch webpage content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }  # Simulate being a browser to not be blocked by website
        response = requests.get(url, headers=headers, timeout=10, verify=False)

        # Detect encoding if not provided by the server
        response.encoding = (
            "utf-8"  # Make sure all characters such as é à etc are correctly rendered
        )

        response.raise_for_status()  # Raise error for bad responses

        # Parse the HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract title and meta description
        title = soup.title.string if soup.title else ""
        meta_desc = ""
        if soup.find("meta", attrs={"name": "description"}):
            meta_desc = soup.find("meta", attrs={"name": "description"}).get("content", "")

        # Extract text from headers and paragraphs
        headers = [h.get_text(strip=True) for h in soup.find_all(["h1", "h2", "h3"])]
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]

        # Combine all extracted text
        extracted_text = f"Description: {meta_desc} \n "
        extracted_text += "\n".join(headers + paragraphs)
        cleaned_text = " ".join(extracted_text.split())
        return title,cleaned_text

    except Exception as e:
        print(f"Error: {e}")
        return "Error"

In [14]:
# Example usage
url = "http://www.oderis.fr"
company_text = scrape_company_text(url)
print(company_text)

('\n      Oderis - Conseil en création de valeur\n', "Description: Cabinet indépendant de conseil financier en transaction et accompagnement opérationnel, pour les fonds d'investissements et dirigeants. Pays/région Pays/région Cabinet indépendanten transaction et accompagnement opérationnel 1erconseil LBOFrance 1erconseil Small CapFrance 2èmeconseil Mid CapFrance 10èmeconseil LBOEurope Oderis portedes valeurs fortes et différenciantessur son marché Entrepreneurship RSE & ESG Business partner Human Focused Nosmétiers Transaction Services Restructuring Transformation Évaluation Tax & Legal ESG 150 Consultants à vos côtés, etheureux de l’être! Oderis enquelques chiffres Collaborateurs Bureaux Associés et directeurs Lignes de service Nos clientsparlent de nous Nosactualités Oderis poursuit sa mue à l’international Oderis poursuit sa mue à l’international Oderis renforce encore son bureau lyonnais Oderis renforce encore son bureau lyonnais Oderis lance son offre ESG avec Radia Benhallam Ode

In [14]:
df["Site internet"]

0                  https://www.anderapartners.com/fr/
1                 https://www.ardian.com/fr/expansion
2                             https://www.astorg.com/
3                         https://www.bridgepoint.eu/
4                                https://capza.co/fr/
5                            https://www.carlyle.com/
6     https://www.rothschildandco.com/en/five-arrows/
7                             https://www.gimv.com/fr
8                             https://ikpartners.com/
9                              https://keensight.com/
10                             https://keensight.com/
11                     https://www.latour-capital.fr/
12                                   https://main.nl/
13                             https://montefiore.eu/
14                     https://www.oakleycapital.com/
15           https://www.paipartners.com/language/en/
16                             https://www.seven2.eu/
17                  https://www.sagard.com/fr/newgen/
18                  https://

In [15]:
df[["Title of website", "Description"]] = df["Site internet"].apply(
    lambda x: scrape_company_text(x) if x is not np.nan else x
).apply(pd.Series)

In [19]:
df

,Fonds,Site internet,Title of website,Description
0,Andera Partners (Mid-cap),https://www.anderapartners.com/fr/,Accueil - ANDERA PARTNERS :ANDERA PARTNERS,Description: Acteur de référencedu Capital-Inv...
1,Ardian (Expansion),https://www.ardian.com/fr/expansion,Investissement dans des PME/ ETI à long terme ...,Description: Ardian soutient le développement ...
2,Astorg (Mid-cap),https://www.astorg.com/,Astorg - Astorg | Uncovering value through the...,Description: Uncovering value through the art ...
3,Bridgepoint (Mid-Cap),https://www.bridgepoint.eu/,Home - Bridgepoint,Description: Bridgepoint is an international a...
4,Capza (Mid-Cap),https://capza.co/fr/,"CAPZA, Capital & Private Debt",Description: CAPZA fournit des financements su...
5,Carlyle (Tech),https://www.carlyle.com/,Carlyle | Global Investment Firm,Description: With $447 billion of assets under...
6,Five Arrows (Principal Investments),https://www.rothschildandco.com/en/five-arrows/,Five Arrows - Rothschild & Co,Description: Learn about Rothschild & Co's Fiv...
7,Gimv,https://www.gimv.com/fr,"Homepage Gimv, Building leading companies | Gimv","Description: Gimv, une société d'investissemen..."
8,IK Partners (Small-Cap),https://ikpartners.com/,Leading European Private Equity Firm. IK Partn...,Description: Hands-on approach to investing in...
9,Keensight (Flagship),https://keensight.com/,Keensight Capital - European Growth Buyout inv...,Description: Keensight Capital is a European G...


In [27]:
df.to_excel("data/resultats.xlsx")

In [21]:
df.loc[df["Homepage Text"] == 'Error','Homepage Text']

63      Error
72      Error
85      Error
87      Error
94      Error
        ...  
1531    Error
1546    Error
1549    Error
1553    Error
1567    Error
Name: Homepage Text, Length: 153, dtype: object

In [22]:
df.to_csv('data/resultats.csv')